In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.io import EventSource
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config
from traitlets.config import Config
from traitlets.config import Config
import time
import pickle


# all muon rings among stereo protons for runs 3978-4000
final_rings = [78124, 78437, 79331, 80266, 81137, 82039, 82691, 84192, 84560, 84631, 86287, 86345, 86590, 86633, 87027, 87511, 87597, 88049, 88103, 88130, 88524, 88598, 88599, 88647, 89104, 89236, 90093, 90324, 90493, 90635, 91012, 91662, 93161, 94747, 96593, 97606, 98657, 99694]
print(len(final_rings))
new_final_rings = final_rings

#for i in final_rings:
#    new_final_rings.append(int(str(39)+str(i)))

print(len(new_final_rings))

In [ ]:
k = 0
other_key = []
for key in final_rings:
    event = event_container[key]
    for tels in event_container[key].trigger.tels_with_trigger:
        #if not np.isnan(event_container[key].muon.tel[tels].efficiency.optical_efficiency):
        other_key.append(key)
        plt.figure(figsize = (9,6))
        camgeom = source.subarray.tel[tels].camera.geometry
        #print("i = ", i, "mse = ",cleaned_event_container[i].muon.tel[tels].parameters.mean_squared_error.to_value())
        #title=f"Event_{i}_optEff{round(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container_protons[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container_protons[i].trigger.tel.keys())}"
        #title=f"Cleaned_IslandRing_{i}_optEff{round(cleaned_event_container[i].muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(cleaned_event_container[i].muon.tel[tels].efficiency.width.to_value(),3)}_triggeredTel{tels}"
        title = f'{key}_k={k}'
        k+=1
        disp = CameraDisplay(camgeom,title=title)
        print(event.dl1.tel[tels].parameters.morphology.n_pixels)
        disp.image = event.simulation.tel[tels].true_image
        #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        #disp.set_limits_percent(95)
        plt.show()

In [ ]:
all_energies = []

for k in range(78,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    print(filename)
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):
        if len(event.trigger.tels_with_trigger) > 1:  
            all_energies.append(event.simulation.shower.energy.to_value())

    

In [ ]:
plt.hist(all_energies, log=True)

In [ ]:
new_proton_energies = [energy**(-0.7) for energy in all_energies]

In [ ]:

sum(new_proton_energies)#/sum(all_energies)

In [ ]:
event_container = {}
all_events = 0
stereo = 0

for k in range(78,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{k}.simtel.gz'
    print(filename)
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = k*1000
    for i,event in enumerate(event_iterator):
        all_events += 1
        if len(event.trigger.tels_with_trigger) > 1:  
            stereo += 1          
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib(event)
            image_processor(event)
            muon_processor(event)
            event_container[keks+i] = event

In [ ]:
# Make Pickle dump with event dict

# save dictionary to person_data.pkl file
with open('/Users/vdk/LST/4LST_stereo_events.pkl', 'wb') as fp:
    pickle.dump(event_container, fp)

## Flagging

In [ ]:
# add time sampling

r1_cut = 2
min_pixel_cut = 50
max_pixel_cut = 200
event_container_flag = {}


for key in event_container:
    
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = np.array([])
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:16]:
                tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut):
                event_container_flag[key] = event_container[key]



# Для меншого значення пікселя віднормувати все
# Для більшого ката спробувати 
# MAx Pixel try to cut  


In [ ]:
# Make Pickle dump with event dict

# save dictionary to person_data.pkl file
#with open('/Users/vdk/LST/4LST_R1Cut2_minPix50.pkl', 'wb') as fp:
#    pickle.dump(event_container_flag, fp)

In [ ]:
# add time sampling
max_time = 14
r1_cut = 2
min_pixel_cut = 40
max_pixel_cut = 200
event_container_flag_2 = {}

threshold_variance = 0.1 # choose events with variance higher than this value

## ADD LIST

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = []
            var_arr = []
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:max_time]:
                tmp_arr.append(np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                var_arr.append(np.var(pix_id))
            
            tmp_arr = np.array(tmp_arr)
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut) and (np.mean(var_arr) > threshold_variance) and (np.mean(var_arr) < 0.4):
                event_container_flag_2[key] = event_container[key]



# Для меншого значення пікселя віднормувати все
# Для більшого ката спробувати   

In [ ]:
# add time sampling
r1_cut = 2
min_pixel_cut = 50
max_pixel_cut = 180
max_time = 14
event_container_flag_3 = {}
threshold_variance = 0.1 # choose events with variance higher than this value

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = []
            var_arr = []
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:max_time]:
                tmp_arr.append(np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                var_arr.append(np.var(pix_id))
            
            tmp_arr = np.array(tmp_arr)
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut) and (np.mean(var_arr) > threshold_variance) and (np.mean(var_arr) < 0.4):
                event_container_flag_3[key] = event_container[key]



# Для меншого значення пікселя віднормувати все
# Для більшого ката спробувати   

In [ ]:
r1_cut = 2
min_pixel_cut = 60
max_pixel_cut = 180
event_container_flag_4 = {}
threshold_variance = 0.1 # choose events with variance higher than this value

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = []
            var_arr = []
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:max_time]:
                tmp_arr.append(np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                var_arr.append(np.var(pix_id))
            
            tmp_arr = np.array(tmp_arr)
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut) and (np.mean(var_arr) > threshold_variance) and (np.mean(var_arr) < 0.4) and (np.mean(var_arr) < 0.4):
                event_container_flag_4[key] = event_container[key]

In [ ]:
r1_cut = 2
min_pixel_cut = 70
max_pixel_cut = 180
event_container_flag_5 = {}

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = []
            var_arr = []
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:max_time]:
                tmp_arr.append(np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                var_arr.append(np.var(pix_id))
            
            tmp_arr = np.array(tmp_arr)

            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut) and (np.mean(var_arr) > threshold_variance) and (np.mean(var_arr) < 0.4):
                event_container_flag_5[key] = event_container[key]

In [ ]:
r1_cut = 2
min_pixel_cut = 80
max_pixel_cut = 200
event_container_flag_6 = {}

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = []
            var_arr = []
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:max_time]:
                tmp_arr.append(np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                var_arr.append(np.var(pix_id))
            
            tmp_arr = np.array(tmp_arr)
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut) and (np.mean(var_arr) > threshold_variance) and (np.mean(var_arr) < 0.4):
                event_container_flag_6[key] = event_container[key]

In [ ]:
r1_cut = 2
min_pixel_cut = 90
max_pixel_cut = 200
event_container_flag_7 = {}

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = []
            var_arr = []
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:max_time]:
                tmp_arr.append(np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                var_arr.append(np.var(pix_id))
            
            tmp_arr = np.array(tmp_arr)
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut) and (np.mean(var_arr) > threshold_variance) and (np.mean(var_arr) < 0.4):
                event_container_flag_7[key] = event_container[key]

In [ ]:
r1_cut = 5
min_pixel_cut = 40
max_pixel_cut = 200
event_container_flag_8 = {}

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = np.array([])
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:16]:
                tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut):
                event_container_flag_8[key] = event_container[key]

In [ ]:
r1_cut = 5
min_pixel_cut = 50
max_pixel_cut = 200
event_container_flag_9 = {}

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = np.array([])
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:16]:
                tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut):
                event_container_flag_9[key] = event_container[key]

In [ ]:
r1_cut = 5
min_pixel_cut = 60
max_pixel_cut = 200
event_container_flag_10 = {}

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = np.array([])
            for pix_id in event_container[key].r1.tel[tel_id].waveform[:,10:16]:
                tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut):
                event_container_flag_10[key] = event_container[key]

In [ ]:
# Створити списки з мюонними подіями і відповідними телескопами
muon_events = []
muon_telescope = []
high_energy = []
k = 1
for i in event_container:
    if i in final_rings:
        for tels in event_container[i].trigger.tels_with_trigger:
            if not np.isnan(event_container[i].muon.tel[tels].efficiency.optical_efficiency):
                #print(k)
                muon_events.append(event_container[i])
                muon_telescope.append(tels)
                print(event_container[i].simulation.shower.energy)
                if event_container[i].simulation.shower.energy.to_value() > 4:
                    high_energy.append(event_container[i])
                    print("HIGH ENERGY = ", i)
                k += 1

print(len(muon_events))


In [ ]:
muon_energies = []
for event in muon_events:
    muon_energies.append(event.simulation.shower.energy.to_value())

x,y,z = plt.hist(muon_energies, bins = 7, label = 'muon energies', histtype = 'step')
plt.xlabel('Energies, TeV')
plt.ylabel('Counts')
plt.legend()
plt.title("Energy spectrum of muon rings")

In [ ]:
plt.figure(figsize = (9,6))
camgeom = source.subarray.tel[tels].camera.geometry
#print("i = ", i, "mse = ",cleaned_event_container[i].muon.tel[tels].parameters.mean_squared_error.to_value())
#title=f"Event_{i}_optEff{round(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container_protons[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container_protons[i].trigger.tel.keys())}"
#title=f"Cleaned_IslandRing_{i}_optEff{round(cleaned_event_container[i].muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(cleaned_event_container[i].muon.tel[tels].efficiency.width.to_value(),3)}_triggeredTel{tels}"
title = f'{k}'
disp = CameraDisplay(camgeom,title=title)
print(high_energy[0].dl1.tel[5].parameters.morphology.n_pixels)
disp.image = event.simulation.tel[tels].true_image
#disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
#disp.set_limits_percent(95)
plt.show()


In [ ]:
# pixel_number_m = []
# k = 0
# for event in muon_events:
#     for tels in event.trigger.tels_with_trigger:
#         if not np.isnan(event.muon.tel[tels].efficiency.optical_efficiency) and event.muon.tel[tels].parameters.mean_squared_error.to_value() < 0.01:
#             pixel_number_m.append(event.dl1.tel[tels].parameters.morphology.n_pixels)
#             # plt.figure(figsize = (9,6))
#             # camgeom = source.subarray.tel[tels].camera.geometry
#             # #print("i = ", i, "mse = ",cleaned_event_container[i].muon.tel[tels].parameters.mean_squared_error.to_value())
#             # #title=f"Event_{i}_optEff{round(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container_protons[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container_protons[i].trigger.tel.keys())}"
#             # #title=f"Cleaned_IslandRing_{i}_optEff{round(cleaned_event_container[i].muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(cleaned_event_container[i].muon.tel[tels].efficiency.width.to_value(),3)}_triggeredTel{tels}"
#             # title = f'{k}'
#             # k+=1
#             # disp = CameraDisplay(camgeom,title=title)
#             # print(event.dl1.tel[tels].parameters.morphology.n_pixels)
#             # disp.image = event.simulation.tel[tels].true_image
#             # #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
#             # disp.cmap = plt.cm.RdBu_r
#             # disp.add_colorbar()
#             # #disp.set_limits_percent(95)
#             # plt.show()

pixel_number_p = []
pixel_r1_number_p = []
high_keys = []
r1_cut = 10

for key in event_container:
    temp_pixel = 0
    temp_r1_pixel = 0
    for tels in event_container[key].trigger.tels_with_trigger:
        tmp_arr = np.array([])

        for pix_id in event_container[key].r1.tel[tels].waveform:
            tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі

        pixel_number = len(tmp_arr[tmp_arr>r1_cut])
        #print(pixel_number)
        if pixel_number > temp_r1_pixel:
            temp_r1_pixel = pixel_number
        if event_container[key].dl1.tel[tels].parameters.morphology.n_pixels > temp_pixel:
            temp_pixel = event_container[key].dl1.tel[tels].parameters.morphology.n_pixels
        if event_container[key].dl1.tel[tels].parameters.morphology.n_pixels > 400:
            high_keys.append(key)
        #print(event_container[key].dl1.tel[tels].parameters.morphology.n_pixels)
    pixel_number_p.append(temp_pixel)
    pixel_r1_number_p.append(temp_r1_pixel)


## Events with large number of pixels

In [ ]:
for key in event_container:
    for tels in event_container[key].trigger.tels_with_trigger:
        if event_container[key].dl1.tel[tels].parameters.morphology.n_pixels > 600:
            plt.figure(figsize = (9,6))
            camgeom = source.subarray.tel[tels].camera.geometry
            #print("i = ", i, "mse = ",cleaned_event_container[i].muon.tel[tels].parameters.mean_squared_error.to_value())
            #title=f"Event_{i}_optEff{round(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container_protons[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container_protons[i].trigger.tel.keys())}"
            #title=f"Cleaned_IslandRing_{i}_optEff{round(cleaned_event_container[i].muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(cleaned_event_container[i].muon.tel[tels].efficiency.width.to_value(),3)}_triggeredTel{tels}"
            title = f'Event_{key}'
            k+=1
            disp = CameraDisplay(camgeom,title=title)
            print("number of dl1 active pixels = ", event_container[key].dl1.tel[tels].parameters.morphology.n_pixels)
            print("number of simulated active pixels = ", np.count_nonzero(event_container[key].simulation.tel[tels].true_image))
            print("energy of shower = ", event_container[key].simulation.shower.energy)
            disp.image = event_container[key].simulation.tel[tels].true_image
            #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
            disp.cmap = plt.cm.RdBu_r
            disp.add_colorbar()
            #disp.set_limits_percent(95)
            plt.show()

### Check number of r1_cut for muon rings

In [ ]:
#histplot(pixel_number_p)
#pixel_number_p
muon_arr=[]
r1_cut = 2
for i,event in enumerate(muon_events):
    #for tels in event.trigger.tels_with_trigger:
        #if not np.isnan(event.muon.tel[tels].efficiency.width) and event.muon.tel[tels].parameters.mean_squared_error.to_value() < 0.01: 

    tmp_arr = np.array([])

    for pix_id in event.r1.tel[muon_telescope[i]].waveform:
        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі

    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
    #print(pixel_number)
    if pixel_number >= 50:
        muon_arr.append(pixel_number)
    #if pixel_number > pixel:
    #    event_container_flag.append(key)
    # if pixel_number > 50:
    #     plt.figure(figsize = (9,6))
    #     camgeom = source.subarray.tel[muon_telescope[i]].camera.geometry
    #     #print("i = ", i, "mse = ",cleaned_event_container[i].muon.tel[tels].parameters.mean_squared_error.to_value())
    #     #title=f"Event_{i}_optEff{round(event_container_protons[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container_protons[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container_protons[i].trigger.tel.keys())}"
    #     #title=f"Cleaned_IslandRing_{i}_optEff{round(cleaned_event_container[i].muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(cleaned_event_container[i].muon.tel[tels].efficiency.width.to_value(),3)}_triggeredTel{tels}"
    #     title = f'Event_{i}'
    #     disp = CameraDisplay(camgeom,title=title)
    #     print("number of dl1 active pixels = ", event.dl1.tel[muon_telescope[i]].parameters.morphology.n_pixels)
    #     print("number of r1_cut  pixels = ", pixel_number)
    #     print("number of simulated active pixels = ", np.count_nonzero(event.simulation.tel[muon_telescope[i]].true_image))
    #     print("energy of shower = ", event.simulation.shower.energy)
    #     disp.image = event.simulation.tel[muon_telescope[i]].true_image
    #     #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
    #     disp.cmap = plt.cm.RdBu_r
    #     disp.add_colorbar()
    #     plt.show()
        
print(muon_arr)

In [ ]:
anomaly_r1 = [i for i in pixel_r1_number_p if i > 300]
new_r1_pixel_p = [i for i in pixel_r1_number_p if i < 300]
overflow_r1_p = [300 for i in anomaly_r1]
new_r1_p = new_r1_pixel_p.copy()
new_r1_p = new_r1_p + overflow_r1_p
len(anomaly_r1)
x,y,z = plt.hist(new_r1_p)

In [ ]:
# Creating overflow bin for proton number of pixels

anomaly = [i for i in pixel_number_p if i > 300]
new_pixel_p = [i for i in pixel_number_p if i < 300]
overflow_p = [300 for i in anomaly]
new_p = new_pixel_p.copy()
new_p = new_p + overflow_p
len(anomaly)
x,y,z = plt.hist(new_p)

In [ ]:
x,y,z = plt.hist(muon_arr, bins = 7,density = True, range=(50,200), label =  'muon r1 cut pixels', histtype='step')
x,y,z = plt.hist(pixel_number_m, bins = 7,density = True, range=(50,200),label = 'muon dl1 pixels', histtype='step')#, alpha = 0.5)
x,y,z = plt.hist(new_p,density = True,alpha = 0.2,label = 'proton dl1 pixels (overflow=300)')
x,y,z = plt.hist(new_r1_p, density = True, alpha = 0.2, label = 'proton r1 pixels (overflow=300)')
print(len(arr))
print(len(pixel_number_m))
plt.legend()
plt.xlabel("Number of pixels")
plt.ylabel("Probability density")
plt.title("Comparison of pixel number in dl1 container and counted as r1 waveform cut")

In [ ]:
x,y,z = plt.hist(muon_arr, bins=range(min(muon_arr), max(muon_arr) + 1, 1),density = True, label =  'muon rings', histtype='step')
#x,y,z = plt.hist(pixel_number_m, bins = 5,density = True, label = 'muon dl1 pixels', histtype='step')#, alpha = 0.5)
#x,y,z = plt.hist(new_p,density = True,alpha = 0.2,label = 'proton dl1 pixels (overflow=300)')
x,y,z = plt.hist(new_r1_p, bins=range(min(new_r1_p), max(new_r1_p) + 1, 1), density = True, alpha = 1, label = 'protons (overflow=300)')
print(len(muon_arr))
print(len(pixel_number_m))
plt.legend()
plt.xlabel("Number of pixels with max R1 amplitude > 2 p.e./ns")
plt.ylabel("Probability density")
plt.title("Pixel number based on R1 waveform cut for muon rings and protons")
print(min(pixel_number_m))

In [ ]:
x,y,z = plt.hist(muon_arr,density = False, label =  'muon rings', histtype='step')
#x,y,z = plt.hist(pixel_number_m, bins = 5,density = True, label = 'muon dl1 pixels', histtype='step')#, alpha = 0.5)
#x,y,z = plt.hist(new_p,density = True,alpha = 0.2,label = 'proton dl1 pixels (overflow=300)')
x,y,z = plt.hist(new_r1_p, density = False, alpha = 1, label = 'protons (overflow=300)')
print(len(muon_arr))
print(len(pixel_number_m))
plt.ylim(1,200)
plt.legend()
plt.xlabel("Number of pixels with max R1 amplitude > 2 p.e./ns")
plt.ylabel("Probability density")
plt.title("Pixel number based on R1 waveform cut for muon rings and protons")
print(min(pixel_number_m))

In [ ]:
pixel = 70
event_container_flag = []
r1_cut = 2

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:

        tmp_arr = np.array([])

        for pix_id in event_container[key].r1.tel[tel_id].waveform:
            tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
        
        pixel_number = len(tmp_arr[tmp_arr>r1_cut])
        #print(pixel_number)
        if pixel_number > pixel:
            event_container_flag.append(key)

    

In [ ]:
count = 0
for key in set(event_container_flag):
    #print("key", key)
    if key in final_rings:
        count+=1

print("survived protons = ", len(set(event_container_flag)))
print("survived rings = ", count)
print("all rings = ", len(final_rings))

In [ ]:
# Видалити з вибірки протони в яких енергія вище ніж 20 ТеВ
pixel = 50
event_container_flag_2 = []
r1_cut = 2

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:

        tmp_arr = np.array([])

        for pix_id in event_container[key].r1.tel[tel_id].waveform:
            tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
        
        pixel_number = len(tmp_arr[tmp_arr>r1_cut])
        #print(pixel_number)
        if pixel_number > pixel and event_container[key].simulation.shower.energy.to_value() < 10:
            event_container_flag_2.append(key)

In [ ]:
r1_cut = 2
min_pixel_cut = 70
max_pixel_cut = 200
event_container_flag = {}

for key in event_container:
    for tel_id in event_container[key].trigger.tels_with_trigger:
            tmp_arr = np.array([])
            for pix_id in event_container[key].r1.tel[tel_id].waveform:
                tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
            pixel_number = len(tmp_arr[tmp_arr>r1_cut])
            
            if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut):
                event_container_flag[key] = event_container[key]


        

In [ ]:
len(event_container_flag)

In [ ]:
###
# Flagging based on R1 Waveforms
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

#pixel_values = [1, 10, 20, 30, 40, 50, 70, 100, 150, 175, 200]
pixel_values = [50]
survived = []
len_container = []
r1_cut = 2


for pixel in pixel_values:
    counter = 0
    event_container_flag = {}

    for k in range(3978,4000):
        filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run{k}.simtel.gz'
        source = EventSource(filename)
        event_iterator = iter(source)
        keks = k*1000
        for i,event in enumerate(event_iterator):
            checker += 1
            if len(event.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
                for tel_id in event.trigger.tels_with_trigger:
                    
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    
                    if pixel_number > pixel:
                        event_container_flag[keks+i] = event

        
    for event in event_container_flag:
        print("thats event = ", event)
        if event in final_rings:
            print("we are really counting")
            counter+=1
            
    survived.append(counter)
    len_container.append(len(event_container_flag))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
# Count R1 pixels in muon rings

len(event_container_flag)
len(survived)

In [ ]:
###
# Flagging based on R1 Waveforms
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

#pixel_values = [1, 10, 20, 30, 40, 50, 70, 100, 150, 175, 200]
pixel_values = [70]
survived_2 = []
len_container_2 = []
r1_cut = 2


for pixel in pixel_values:
    counter = 0
    event_container_flag_2 = {}

    for k in range(3978,4000):
        filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run{k}.simtel.gz'
        source = EventSource(filename)
        event_iterator = iter(source)
        keks = k*1000
        for i,event in enumerate(event_iterator):
            checker += 1
            if len(event.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
                for tel_id in event.trigger.tels_with_trigger:
                    
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    
                    if pixel_number > pixel:
                        event_container_flag_2[keks+i] = event

        
    # for event in event_container_flag:
    #     print("thats event = ", event)
    #     if event in final_rings:
    #         print("we are really counting")
    #         counter+=1
            
    survived_2.append(counter)
    len_container_2.append(len(event_container_flag))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
len(event_container_flag)

# Reweight events

### Muons

In [ ]:
event_container_flag_rw = event_container_flag_7.copy()
print(len(event_container_flag_rw))
# make list with survived muons
survived_muons = []
for event in event_container_flag_rw:
    if event in new_final_rings:
        survived_muons.append(event)

#print(event_container_flag_rw)
# Find energy weights of muons
rescaling_muons = []
muon_energies = []
for key in event_container_flag_rw:
    if key in survived_muons:
        rescaling_muons.append((event_container_flag_rw[key].simulation.shower.energy.to_value())**(-0.7))
        muon_energies.append(event_container_flag_rw[key].simulation.shower.energy)


print(sorted(muon_energies, reverse=True))
print(sorted(rescaling_muons))

print(sum(rescaling_muons))

rescaled_sum_m = sum(rescaling_muons)

### Protons

In [ ]:
rescaling_protons = []
proton_energies = []
# Find energy weights of protons
for key in event_container_flag_rw:
    if key not in survived_muons:
        rescaling_protons.append((event_container_flag_rw[key].simulation.shower.energy.to_value())**(-0.7))
        proton_energies.append(event_container_flag_rw[key].simulation.shower.energy)


print(sorted(proton_energies, reverse=True))
print(sorted(rescaling_protons))

print(sum(rescaling_protons))
rescaled_sum_p = sum(rescaling_protons)

### Comparing of rescaled flagged events

In [ ]:
print("==================================================")
print("Initial number of survived muons = ", len(survived_muons))
print("Initial number of survived protons = ", len(event_container_flag_rw))
print("==================================================")
print("Rescaled number of survived muons = ", round(rescaled_sum_m))
print("Rescaled number of survived protons = ", round(rescaled_sum_p))
print("==================================================")

In [ ]:
fadsf

In [ ]:
###
# Flagging based on R1 Waveforms
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

#pixel_values = [1, 10, 20, 30, 40, 50, 70, 100, 150, 175, 200]
pixel_values = [50]
survived_3 = []
len_container_3 = []
r1_cut = 2


for pixel in pixel_values:
    counter = 0
    event_container_flag_3 = {}

    for k in range(3978,4000):
        filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run{k}.simtel.gz'
        source = EventSource(filename)
        event_iterator = iter(source)
        keks = k*1000
        for i,event in enumerate(event_iterator):
            checker += 1
            if len(event.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
                for tel_id in event.trigger.tels_with_trigger:
                    
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    
                    if pixel_number > pixel:
                        event_container_flag_3[keks+i] = event

        
    # for event in event_container_flag:
    #     print("thats event = ", event)
    #     if event in final_rings:
    #         print("we are really counting")
    #         counter+=1
            
    survived_3.append(counter)
    len_container_3.append(len(event_container_flag_3))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
raw_rings = {78124, 78437, 80266, 82039, 82691, 84161, 84192, 84631, 86287, 86345, 86633, 87027, 87511, 87597, 88049, 88103, 88130, 88524, 88598, 88599, 88647, 89104, 89236, 90324, 90493, 90635, 91012, 91662, 94747, 96593, 98657, 99076, 99694}
cleaned_rings = {78437, 79331, 80266, 80417, 81137, 82691, 84192, 84560, 84631, 86287, 86345, 86590, 86633, 87027, 87597, 88049, 88103, 88130, 88524, 88599, 89104, 89236, 90093, 90324, 90493, 91662, 93161, 94747, 96593, 97606, 99694}
all_rings = raw_rings.union(cleaned_rings)

In [ ]:
print(sorted(all_rings))
print(len(all_rings))

In [ ]:
# HISTPLOT FOR ALL CASES PIXEL NUMBER in muons and protons

r1_cut = 2

pure_muons = [2414, 3211, 3403, 6317, 6466, 7418, 8234, 8448, 9404, 9433, 12262, 14402, 15405, 19103, 20229, 20402, 21288, 21433, 22205, 22271, 22390, 26113, 26164, 26347, 26365, 27176, 27374, 28000, 28328, 29266, 29390, 30405, 30462, 31176, 31386, 32064, 32298, 33057, 33071, 35052, 35375, 36231, 36452, 37363, 39094, 39376]
pixel_number_arr_m_pure = []

for key in muon_events:
    if key in pure_muons:
        temp_arr = []
        for tel in event.trigger.tels_with_trigger:
            if not np.isnan(muon_events[key].muon.tel[tel].efficiency.optical_efficiency) and muon_events[key].muon.tel[tel].efficiency.optical_efficiency < 0.26:
                #print("key = ", key, "tel = ", tel)
                #print("optical eff = ", muon_events[key].muon.tel[tel].efficiency.optical_efficiency)
                pixel_mask = np.array(muon_events[key].simulation.tel[tel].true_image, dtype = bool)
                temp_event = muon_events[key].r1.tel[tel].waveform[pixel_mask, :]
                for i in temp_event:
                    temp_arr.append(np.max(i))
                counter += len(temp_arr)
                hist = np.histogram(temp_arr, bins = 100)
                mask = np.array(np.where(hist[1] > r1_cut)[0])
                pixel_number_arr_m_pure.append(sum(hist[0][mask[:-1]]))        
                    
# Protons pixel number


pixel_number_arr_p_pure = []
k = 1
filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
source = EventSource(filename, max_events=30)
event_iterator = iter(source)
for event in event_iterator:
    for tel in event.trigger.tels_with_trigger:
        temp_arr = []
        pixel_mask = np.array(event.simulation.tel[tel].true_image, dtype = bool)
        temp_event = event.r1.tel[tel].waveform[pixel_mask, :]
        for i in temp_event:
            temp_arr.append(np.max(i))
        hist = np.histogram(temp_arr, bins = 100)
        mask = np.array(np.where(hist[1] > r1_cut)[0])
        pixel_number_arr_p_pure.append(sum(hist[0][mask[:-1]]))


# pixel_number_arr_nsb = []
# nsb_scale = 10
# filename = f'/Users/vdk/Dummy100_nsb_scaling_factor_{nsb_scale}.simtel.gz'
# source = EventSource(filename, max_events=30)
# event_iterator = iter(source)
# for event in event_iterator:
#     temp_arr = []  
#     #print("t", event.r1.tel[1].waveform)
#     for pixel in event.r1.tel[1].waveform:
#         #print("here")
#         temp_arr.append(np.max(pixel))
#         #print(np.max(pixel))
#         #print("nsb max = ", np.max(i))
#     hist = np.histogram(temp_arr, bins = 100, range = (0,15))
#     mask = np.array(np.where(hist[1] > r1_cut)[0])
#     #print(np.where(hist[1] > r1_cut))
#     pixel_number_arr_nsb.append(sum(hist[0][mask[:-1]]))
    
q,w,e = plt.hist(pixel_number_arr_m_pure, bins = 4, alpha = 0.5, label = 'pure muons', density = True)
r,t,y = plt.hist(pixel_number_arr_p_pure, bins = 4, alpha = 0.5, label = 'pure protons', density = True)
#r,t,y = plt.hist(pixel_number_arr_nsb, bins = 4, alpha = 0.5, label = 'nsb', density = True)

plt.legend()
plt.xlabel(f"Number of pixels with max R1 amplitude > {r1_cut} p.e./ns")
plt.ylabel("Probability mass function")
plt.grid()
print(len(pixel_number_arr_m_pure))
print(len(pixel_number_arr_p_pure))
print(pixel_number_arr_nsb)

In [ ]:
event_container